In [1]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px

# Load dataset
df = pd.read_csv('clothing_sales.csv', parse_dates=['date'])

In [8]:
# Initialize the Dash app
app = dash.Dash(__name__)
app.title = "Clothing Sales Dashboard"

# Unique categories for dropdown
categories = df['category'].unique()

# Update the layout to include the new graph
app.layout = html.Div(style={'fontFamily': 'Arial, sans-serif', 'margin': '20px'}, children=[
    html.H1("Clothing Sales Dashboard", style={'textAlign': 'center'}),
    
    # Filters
    html.Div([
        html.Div([
            html.Label("Select Category"),
            dcc.Dropdown(
                id='category-dropdown',
                options=[{'label': cat, 'value': cat} for cat in categories],
                value=list(categories),
                multi=True
            )
        ], style={'width': '48%', 'display': 'inline-block'}),
        
        html.Div([
            html.Label("Select Date Range"),
            dcc.DatePickerRange(
                id='date-picker',
                start_date=df['date'].min(),
                end_date=df['date'].max(),
                display_format='YYYY-MM'
            )
        ], style={'width': '48%', 'display': 'inline-block', 'float': 'right'}),
    ], style={'padding': '10px', 'backgroundColor': '#f9f9f9', 'borderRadius': '5px'}),
    
    # Charts
    html.Div([
        dcc.Graph(id='revenue-line-chart'),
        dcc.Graph(id='top-categories-bar'),
        dcc.Graph(id='price-units-scatter'),
        dcc.Graph(id='sales-heatmap'),
        dcc.Graph(id='category-pie-chart'),
        dcc.Graph(id='animated-scatter'),  # Add the new graph component
    ], style={'marginTop': '20px'})
])

# Update the callback to include the new output
@app.callback(
    [
        Output('revenue-line-chart', 'figure'),
        Output('top-categories-bar', 'figure'),
        Output('price-units-scatter', 'figure'),
        Output('sales-heatmap', 'figure'),
        Output('category-pie-chart', 'figure'),
        Output('animated-scatter', 'figure'),  # Add new output
    ],
    [
        Input('category-dropdown', 'value'),
        Input('date-picker', 'start_date'),
        Input('date-picker', 'end_date')
    ]
)
def update_charts(selected_categories, start_date, end_date):
    # Filter data
    mask = (
        df['category'].isin(selected_categories) &
        (df['date'] >= pd.to_datetime(start_date)) &
        (df['date'] <= pd.to_datetime(end_date))
    )
    dff = df.loc[mask]
    
    # Color scheme
    colorscale = px.colors.qualitative.G10

     # 1. Line chart: Total Revenue Over Time by Category
    revenue_time_cat = (
        dff.groupby(['date', 'category'])['revenue']
           .sum()
           .reset_index()
    )
    fig1 = px.line(
        revenue_time_cat,
        x='date', y='revenue', color='category',
        title='Revenue Over Time by Category',
        labels={'date':'Date','revenue':'Revenue','category':'Category'}
    )

    # 2. Bar chart: Revenue by Category (3D)
    cat_revenue = (
        dff.groupby('category')['revenue']
           .sum()
           .reset_index()
           .sort_values('revenue', ascending=False)
    )
    fig2 = px.bar(
        cat_revenue,
        x='category', y='revenue',
        title='Total Revenue by Category',
        labels={'revenue':'Revenue ($)','category':'Category'},
        color='revenue',
        color_continuous_scale=px.colors.sequential.Viridis,
        template='plotly_white'
    )
    fig2.update_layout(
        plot_bgcolor='rgba(240,240,240,0.2)',
        paper_bgcolor='rgba(0,0,0,0)',
        font=dict(family="Arial", size=12),
        coloraxis_showscale=True,
        xaxis=dict(tickangle=30),
        yaxis=dict(showgrid=True, gridcolor='rgba(200,200,200,0.4)')
    )
    fig2.update_traces(marker=dict(line=dict(width=1, color='rgba(0,0,0,0.4)')))

 # 3. Scatter: Price vs. Units Sold
    sample = dff.sample(min(len(dff), 500))
    fig3 = px.scatter(
        sample,
        x='price', y='units_sold',
        color='category',
        title='Price vs. Units Sold',
        hover_data=['sku'],
        labels={'price':'Price','units_sold':'Units Sold','category':'Category'}
    )

    # 4. Heatmap: Monthly Revenue by Category (enhanced)
    heatmap_df = (
        dff.groupby([dff['date'].dt.to_period('M').dt.to_timestamp(), 'category'])['revenue']
           .sum()
           .reset_index()
           .pivot(index='date', columns='category', values='revenue')
           .fillna(0)
    )
    fig4 = px.imshow(
        heatmap_df.T,
        labels={'x':'Month', 'y':'Category', 'color':'Revenue ($)'},
        x=heatmap_df.index.astype(str),
        y=heatmap_df.columns,
        title='Monthly Revenue Heatmap',
        color_continuous_scale=px.colors.sequential.Plasma
    )
    fig4.update_layout(
        font=dict(family="Arial", size=12),
        paper_bgcolor='rgba(0,0,0,0)',
        margin=dict(l=50, r=50, t=50, b=50)
    )
    fig4.update_xaxes(tickangle=45, side="bottom")
    fig4.update_traces(hoverongaps=False)

    # 5. Pie chart: Revenue Distribution by Category (3D Donut)
    total_rev = cat_revenue
    fig5 = px.pie(
        total_rev,
        names='category',
        values='revenue',
        title='Revenue Distribution by Category',
        color_discrete_sequence=px.colors.qualitative.Bold,
        hole=0.4
    )
    fig5.update_layout(
        font=dict(family="Arial", size=12),
        paper_bgcolor='rgba(0,0,0,0)',
        legend=dict(orientation="h", yanchor="bottom", y=-0.2, xanchor="center", x=0.5),
        margin=dict(l=20, r=20, t=60, b=100)
    )
    fig5.update_traces(
        textinfo='percent+label',
        textposition='inside',
        marker=dict(line=dict(color='#FFFFFF', width=2))
    )

    # 6. Animated scatter: revenue vs units_sold over time
    anim_df = (
        dff.groupby(['date', 'category'])
           .agg({'units_sold':'sum','revenue':'sum'})
           .reset_index()
    )
    fig6 = px.scatter(
        anim_df,
        x='units_sold', y='revenue',
        animation_frame=anim_df['date'].dt.strftime('%Y-%m'),
        animation_group='category',
        color='category',
        size='revenue',
        hover_name='category',
        title='Animated: Units Sold vs Revenue Over Time',
        labels={'units_sold':'Units Sold','revenue':'Revenue ($)','animation_frame':'Month'},
        color_discrete_sequence=colorscale
    )
    fig6.update_layout(
        plot_bgcolor='rgba(240,240,240,0.2)',
        paper_bgcolor='rgba(0,0,0,0)',
        font=dict(family="Arial", size=12),
        xaxis=dict(showgrid=True, gridcolor='rgba(200,200,200,0.4)'),
        yaxis=dict(showgrid=True, gridcolor='rgba(200,200,200,0.4)'),
        transition={'duration':500}
    )

    return fig1, fig2, fig3, fig4, fig5, fig6

In [9]:
if __name__ == '__main__':
    app.run(debug=True)